# May 24 final review


## cruise control

problem from: https://ctms.engin.umich.edu/CTMS/index.php?example=CruiseControl&section=SystemModeling#:~:text=The%20purpose%20of%20the%20cruise,according%20to%20a%20control%20law.

![cruiseCtl](https://drive.google.com/uc?id=1nUmXR2yx4QRbMvJDuB7BrmXScMVc3BiN)

$$m\dot{v}+bv=u$$

$$y=v$$

$m=1000$ kg (vehicle mass)

$b = 50$ Ns/m (dampening coefficient)

## week 6: state space control

### 6a. state feedback

- pole placement / eigenvalue assignment -- how to pick? 
- determine shape of $K$
- determine values of $K$ symbolically and numerically 

- $\lambda=\sigma\pm j\omega,a(s)=s^2+2j\sigma\omega+(\omega^2+\sigma^2)$ (pole placement)
- $\lambda=-\zeta\omega\pm \omega\sqrt{\zeta^2-1},a(s)=s^2+2\zeta\omega s+\omega^2$ (canonical second order system)

### 6b. state estimation

- pole placement / eigenvalue assignment -- how to pick? 
- determine shape of $L$
- determine values of $L$ symbolically and numerically 

### 6c. stabilizing controller

- why do we do this?

### 6c. stabilizing controller (nonlinear)

- why do we do this?

## week 7: frequency analysis

### 7a. state space to transfer function

$$G_{yu}(s)=C(sI-A)^{-1}B+D$$

- algebraically or computationally

### 7b. bode plots

### 7c effect of disturbances

can't simultaneously reject disturbances on input and output at the same time
- sensitivity $G_{yw}=S(j\omega)=\frac{1}{1+PC}$, e.g. sensor noise
- and complementary sensitivity $-G_{uv} = T(j\omega)=\frac{PC}{1+PC}$, e.g. wind pushing on car, hill

can't reshape $S$ to whatever you desire - Bode integral formula / principle
- any range of frequencies where controller attenuates disturbance is compensated by a range where controller amplifies disturbance

## week 8: frequency control

### 8a. Nyquist stability criterion

if $L = PC$ has no poles in the right-half plane, then $\frac{L}{1+L}$ is stable $\leftrightarrow$ $\Omega$ does not encircle $-1\in\mathbb{C}$

Nyquist plot: $\Omega=\{L(j\omega)\in\mathbb{C}: -\infty<\omega<\infty\}$

### 8b. gain, phase, and stability margins

* *stability margin* $s_m$ = distance from $\Omega$ to $-1\in\mathbb{C}$;
* *gain margin* $g_m$ = smallest increase in gain that causes $\Omega$ to encircle $-1$ (computed as the inverse of distance from $\Omega$ to $-1\in\mathbb{C}$);
* *phase margin* $\varphi_m$ = smallest change in phase that causes $\Omega$ to encircle $-1$ (computed as the distance from $\Omega$ to $-1\in\mathbb{C}$ restricted to rotation of $\Omega$).

### 8c. root locus

general idea: for a causal plant $P$ with proportional feedback controller $C(s) = K$, the closed-loop system $\frac{PC}{1+PC}$ will approximately behave like the poles of $P(s)$ at small $K$s and more like the zeros of $P(s)$ at large $K$s. 

- this will not be on the exam